# Assignment 7: Linear Model Selection and Regularization

Dataset(s) needed:
Kaggle House Prices (https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities    ...     PoolArea PoolQC Fence MiscFeature MiscVal  \
0         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
1         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
2         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
3         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
4         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   

  MoSold YrSold  SaleType  SaleCondition  SalePrice  
0      2   2008        WD         Normal     208500  
1      5   2007        WD         Normal     181500  
2      9   2008        WD         Normal     223500  
3      2   2006        WD        Abnorml     140000  
4     12   2008        WD         Normal     250000  

[5 rows x 81 columns]

#### Drop the Id columns from both train and test as these are not needed for prediction

In [3]:
train = train.drop('Id',axis=1)
test = test.drop('Id',axis=1)

#### Visualize a scatter plot of 'GrLivArea' in the x-axis and 'SalePrice' in the y-axis. Can you spot any outliers?

In [4]:
import matplotlib.pyplot as plt

x = train['GrLivArea']
y = train['SalePrice']

plt.scatter(x,y)
plt.ylabel('SalePrice')
plt.xlabel('GrLivArea')
plt.show()

# There appears to be a handful of outliers at x values higher than ~4000. Only about four though.

#### Remove outliers in the data for all GrLivArea greater than 4000 then check the scatter plot again

In [5]:
# Remove outliers for train['GrLivArea']>4000
train = train[train['GrLivArea']<4000]

x = train['GrLivArea']
y = train['SalePrice']

plt.scatter(x,y)
plt.ylabel('SalePrice')
plt.xlabel('GrLivArea')
plt.show()

#### Convert categorical variable into dummy variables using pandas get_dummies API


In [6]:
train = pd.get_dummies(train)

#### Impute missing data by the mean of each column.

In [7]:
train = train.fillna(train.mean())

Generate train test split 70/30

In [8]:
from sklearn.model_selection import train_test_split

target =  'SalePrice'
features = list(train.drop(target,axis=1).columns.values)

X = train[features]
y = train[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

## Linear Regression

In [17]:
from sklearn.linear_model import Ridge, ElasticNet, LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from numpy import sqrt

#Fit a linear regression model to this data
lr = LinearRegression()
lr = lr.fit(X_train,y_train)
y_preds = lr.predict(X_test)

lr_mse = mean_squared_error(y_test,y_preds)
lr_rmse = sqrt(lr_mse)

#Compute the RMSE 
print("RMSE:",lr_rmse.round(2))

RMSE: 23841.83


## Lasso Regression

#### Fit lasso regression to training data with alpha 20. (Bonus: You could try CV to find the best alpha within a range)

In [18]:
from sklearn import linear_model

alpha = 20

lasso = linear_model.Lasso(alpha=alpha)
lasso = lasso.fit(X_train,y_train)
#Compute the RMSE 
y_preds = lasso.predict(X_test)

lasso_mse = mean_squared_error(y_test,y_preds)
lasso_rmse = sqrt(lasso_mse)

#Compute the RMSE 
print("Alpha: {}  -  RMSE: {}".format(alpha,lasso_rmse.round(2)))

Alpha: 20  -  RMSE: 22426.0


#### How many columns has the model chosen?

In [19]:
lcs = lasso.coef_
len([n for n in lcs if n > 0])

93

## Ridge Regression

#### Perform ridge regression with alpha set to 0.1 and normalize to True. (Bonus: You could try CV to find the best alpha within a range). 

In [20]:
from sklearn.linear_model import Ridge

alpha = .1

rij = linear_model.Ridge(alpha=alpha,normalize=True)
rij = rij.fit(X_train,y_train)
#Compute the RMSE 
y_preds = rij.predict(X_test)



Alpha: 0.1  -  RMSE: 21552.61


#### What is your root mean squared error (RMSE)? Comment on your observation.

In [21]:
#Compute the RMSE 
rij_mse = mean_squared_error(y_test,y_preds)
rij_rmse = sqrt(rij_mse)

print("Alpha: {}  -  RMSE: {}".format(alpha,rij_rmse.round(2)))

Alpha: 0.1  -  RMSE: 21552.61


## Elastic Net Regression

#### Perform elastic net regression with different values of alpha (0.01, 0.1 and 20). Compute the RMSE as well. 


In [22]:
from sklearn.linear_model import ElasticNet

def compute_EN(alpha):
    en = ElasticNet(alpha=alpha)
    en = en.fit(X_train,y_train)
    preds = en.predict(X_test)
    mse = mean_squared_error(y_test,preds)
    rmse = sqrt(mse)
    print('Alpha:{} - RMSE:{}'.format(alpha,rmse.round(2)))

alphas = [.01,.1,20]

for a in alphas:
    compute_EN(a)

Alpha:0.01 - RMSE:21776.88
Alpha:0.1 - RMSE:22230.14
Alpha:20 - RMSE:33106.36
